<a href="https://colab.research.google.com/github/Manvi190502/AgenticAI/blob/main/OpenAI_multiagent_partial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

OpenAi supports multiagent not groq.

In [ ]:
!pip install langgraph-supervisor langchain-openai langchain_groq

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq
from google.colab import userdata

model=ChatOpenAI(temperature=0, api_key="" , model="gpt-4o-mini")

'''
api_key = userdata.get("GROQ_API_KEY")

model = ChatGroq(
    model="llama-3.3-70b-versatile",
    groq_api_key=api_key,
    temperature=0
)
'''

In [ ]:
!pip install duckduckgo-search ddgs langchain_community langchain langgraph

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

def search_duckduckgo(query: str):
  """Searches DuckDuckGo using Langchain's DuckDuckGoSearchRun tool."""
  search=DuckDuckGoSearchRun()
  return search.invoke(query)

result = search_duckduckgo("What is the capital of France?")
print(result)

In [ ]:
def add(a:float, b:float)-> float:
  """Add two numbers."""
  return a+b

def multiply(a:float, b:float)-> float:
  """Multiply two numbers."""
  return a*b

In [ ]:
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent

math_agent = create_react_agent(
    model=model,
    tools=[add,multiply],
    name="math_expert",
    prompt="You are a math expert. Always use one tool at a time."
)

In [ ]:
research_agent = create_react_agent(
    model=model,
    tools=[search_duckduckgo],
    name="research_expert",
    prompt="You are a world class researcher with access to web earch. Do not do any math."
)

In [ ]:
#create supervisor workflow
workflow= create_supervisor(
    [research_agent, math_agent],
    model=model,
    prompt=(
        "You are a team supervisor managing a research expert and a math expert."
        "For current events, use research_agent. "
        "For math problems, use math_agent."
    )
)

In [ ]:
app=workflow.compile()

result = app.invoke({
    "messages": [
        {
        "role":"user",
        "content": "What is quantum computing?"
        }
    ]
})
result

In [ ]:
for m in result["messages"]:
  m.pretty_print()

In [ ]:
app=workflow.compile()

result = app.invoke({
    "messages": [
        {
        "role":"user",
        "content": "What is the weather of delhi today. Multiply it by 2 and add 5"
        }
    ]
})

for m in result["messages"]:
  m.pretty_print()